## Merging and prerpocessing of NHANES dataset for every individual.

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Set directory path
import os
os.chdir("/content/drive/My Drive/COMP-5800-YDK-2020_Group1")

In [3]:
import pandas as pd
import numpy as np
import glob
import operator

import matplotlib.pyplot as plt
import seaborn as sns

Function to merge the sectional Files

In [4]:
#/** Function for merging the datasets.
#   filenames - names for individual file in a particular folder.
#**/
def merge_file(filenames):
    finalmergeddf = pd.DataFrame()
    for filename in filenames:
        readdf = pd.read_sas(filename, encoding = 'utf-8')
        # If there is no sequence number column in file, avoid that file from merging.
        if 'SEQN' in readdf.columns:
            if finalmergeddf.empty:
              finalmergeddf = readdf
            else:
              finalmergeddf = pd.merge(finalmergeddf, readdf, how = 'outer')

    return finalmergeddf

Demographics Data

In [5]:
df_demographics = pd.read_sas('Data/Demographics/DEMO_J.XPT')

In [6]:
df_demographics.drop(df_demographics.columns[df_demographics.isnull().sum() > 500], axis=1, inplace=True)

In [7]:
df_demographics.shape

(9254, 30)

In [8]:
df_demographics.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,DMDBORN4,DMDCITZN,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2
0,93703.0,10.0,2.0,2.0,2.0,5.0,6.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,3.000000e+00,5.397605e-79,5.397605e-79,1.0,2.0,3.0,1.0,9246.491865,8539.731348,2.0,145.0,15.0,15.0
1,93704.0,10.0,2.0,1.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,4.0,4.0,2.000000e+00,5.397605e-79,5.397605e-79,1.0,2.0,3.0,1.0,37338.768343,42566.614750,1.0,143.0,15.0,15.0
2,93705.0,10.0,2.0,2.0,66.0,4.0,4.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,5.397605e-79,5.397605e-79,1.000000e+00,2.0,4.0,1.0,2.0,8614.571172,8338.419786,2.0,145.0,3.0,3.0
3,93706.0,10.0,2.0,1.0,18.0,5.0,6.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN,NaN,5.0,5.0,5.397605e-79,5.397605e-79,1.000000e+00,1.0,4.0,3.0,1.0,8548.632619,8723.439814,2.0,134.0,NaN,NaN
4,93707.0,10.0,2.0,1.0,13.0,5.0,7.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,7.0,7.0,5.397605e-79,3.000000e+00,5.397605e-79,1.0,3.0,2.0,1.0,6769.344567,7064.609730,1.0,138.0,10.0,10.0


Dietary Data

In [9]:
filenames_dietary = [i for i in glob.glob('Data/Dietary/DSQ*.{}'.format('XPT'))]

In [10]:
df_dietary = merge_file(filenames_dietary)

In [11]:
df_dietary = df_dietary.groupby('SEQN').max().reset_index()

In [12]:
null_dietCol = df_dietary.columns[df_dietary.columns.str.contains(r'DSQ128|DSD128')].tolist()
df_dietary[null_dietCol] = df_dietary[null_dietCol].fillna(0)

In [13]:
df_dietary.drop(df_dietary.columns[df_dietary.isnull().sum() > 500], axis=1, inplace=True)

In [14]:
df_dietary.shape

(9254, 44)

In [15]:
df_dietary.head()

,SEQN,DSDCOUNT,DSDANCNT,DSD010,DSD010AN,DSQ128A,DSQ128B,DSQ128C,DSQ128D,DSQ128E,DSQ128F,DSQ128G,DSQ128H,DSQ128I,DSQ128J,DSQ128K,DSQ128L,DSQ128M,DSQ128N,DSQ128O,DSQ128P,DSQ128Q,DSQ128R,DSD128T,DSD128V,DSD128W,DSD128X,DSD128Z,DSD128AA,DSD128BB,DSD128DD,DSD128EE,DSD128FF,DSD128GG,DSD128HH,DSD128II,DSD128JJ,DSD128KK,DSD128LL,DSD128MM,DSQ128NN,DSD128OO,DSD128PP,DSQ128S
0,93703.0,1.000000e+00,5.397605e-79,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,93704.0,5.397605e-79,5.397605e-79,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,93705.0,8.000000e+00,5.397605e-79,1.0,2.0,10.0,0.0,0.0,13.0,14.0,0.0,16.0,17.0,18.0,0.0,20.0,0.0,22.0,0.0,24.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,93706.0,5.397605e-79,5.397605e-79,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,93707.0,5.397605e-79,5.397605e-79,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Examination Data

In [16]:
filenames_examination = [i for i in glob.glob('Data/Examination/*.{}'.format('XPT'))]

In [17]:
df_examination = merge_file(filenames_examination)

In [18]:
df_examination['BPXCHR'].fillna(df_examination['BPXPLS'], inplace=True)

In [19]:
df_examination.drop(['BPXPLS'], axis=1, inplace=True)

In [20]:
df_examination.drop(df_examination.columns[df_examination.isnull().sum() > 500], axis=1, inplace=True)

In [21]:
df_examination.dtypes

SEQN        float64
BPXCHR      float64
BPXPULS     float64
BMDSTATS    float64
BMXWT       float64
             ...   
OHX21SE      object
OHX28SE      object
OHX29SE      object
OHX30SE      object
OHX31SE      object
Length: 138, dtype: object

In [22]:
df_examination.shape

(8704, 138)

In [23]:
df_examination.head()

,SEQN,BPXCHR,BPXPULS,BMDSTATS,BMXWT,OHDEXSTS,OHDRCSTS,OHDDESTS,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,OHX11CTC,OHX12CTC,OHX13CTC,OHX14CTC,OHX15CTC,OHX18CTC,OHX19CTC,OHX20CTC,OHX21CTC,OHX22CTC,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC,OHX02CSC,OHX03CSC,OHX04CSC,OHX05CSC,...,OHX08RSC,OHX09RSC,OHX10RSC,OHX11RSC,OHX12RSC,OHX13RSC,OHX14RSC,OHX15RSC,OHX18RSC,OHX19RSC,OHX20RSC,OHX21RSC,OHX22RSC,OHX23RSC,OHX24RSC,OHX25RSC,OHX26RSC,OHX27RSC,OHX28RSC,OHX29RSC,OHX30RSC,OHX31RSC,OHX02SE,OHX03SE,OHX04SE,OHX05SE,OHX07SE,OHX10SE,OHX12SE,OHX13SE,OHX14SE,OHX15SE,OHX18SE,OHX19SE,OHX20SE,OHX21SE,OHX28SE,OHX29SE,OHX30SE,OHX31SE
0,93703.0,120.0,1.0,1.0,13.7,1.0,1.0,1.0,U,U,D,D,D,D,D,D,D,D,D,D,U,U,U,U,D,D,D,D,D,D,D,D,D,D,U,U,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,93704.0,114.0,1.0,1.0,13.9,1.0,1.0,1.0,U,U,D,D,D,D,D,D,D,D,D,D,U,U,U,U,D,D,D,D,D,D,D,D,D,D,U,U,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,93705.0,52.0,1.0,1.0,79.5,1.0,1.0,1.0,F,E,F,F,S,S,S,Y,S,S,F,F,F,F,F,R,F,F,S,S,S,S,S,S,F,F,R,F,1,,1,1,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,93706.0,82.0,1.0,1.0,66.3,1.0,1.0,1.0,S,F,S,S,S,S,S,S,S,S,S,S,S,S,S,F,S,S,S,S,S,S,S,S,S,S,S,S,,1,,,...,,,,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,93707.0,100.0,1.0,1.0,45.4,1.0,1.0,1.0,U,S,D,D,D,S,S,S,S,D,D,D,S,U,U,S,D,S,S,S,S,S,S,S,D,S,S,S,,,,,...,,,,,,,,,,,,,,,,,,,,,,,9,0,0,0,0,0,0,0,0,9,9,0,0,9,0,9,0,9


Laboratory Data

In [24]:
filenames_laboratory = [i for i in glob.glob('Data/Laboratory/*.{}'.format('XPT'))]

In [25]:
df_laboratory = merge_file(filenames_laboratory)

In [26]:
df_laboratory = df_laboratory.groupby('SEQN').max().reset_index()

In [27]:
df_laboratory.shape

(8366, 190)

In [28]:
df_laboratory.head()

,SEQN,URXUMA,URXUMS,URDUMALC,URXUCR,URXCRS,URDUCRLC,URDACT,LBDHDD,LBDHDDSI,LBXTC,LBDTCSI,LBXBCR,LBDBCRSI,LBDBCRLC,LBXBCO,LBDBCOSI,LBDBCOLC,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBDBANO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXMC,LBXRDW,LBXPLTSI,LBXMPSI,LBXNRBC,LBXCOT,...,LBXSGB,LBDSGBSI,LBXSGL,LBDSGLSI,LBXSGTSI,LBDSGTLC,LBXSIR,LBDSIRSI,LBXSLDSI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSKSI,LBXSNASI,LBXSTB,LBDSTBSI,LBDSTBLC,LBXSCA,LBDSCASI,LBXSCH,LBDSCHSI,LBXSTP,LBDSTPSI,LBXSTR,LBDSTRSI,LBXSUA,LBDSUASI,URXVOL1,URDFLOW1,URDTIME1,URXVOL2,URDFLOW2,URDTIME2,URXVOL3,URDFLOW3,URDTIME3,URXPREG,LBXVIC,LBDVICSI,LBDVICLC
0,93703.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.4,47.8,8.0,42.6,1.0,0.7,3.5,0.6,3.2,0.1,1.000000e-01,4.25,13.1,37.0,87.0,30.8,35.4,12.8,239.0,8.6,1.000000e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93705.0,3.2,3.2,5.397605e-79,24.0,2121.6,5.397605e-79,13.33,60.0,1.55,157.0,4.06,0.29,5.58,1.0,0.2,3.39,5.397605e-79,8.6,40.0,7.4,48.8,2.9,1.0,3.4,0.6,4.2,0.2,1.000000e-01,5.48,11.9,36.7,67.0,21.7,32.4,15.6,309.0,7.9,5.397605e-79,0.028,...,2.9,29.0,85.0,4.72,21.0,5.397605e-79,94.0,16.8,174.0,280.0,4.0,1.292,4.0,141.0,0.6,10.26,5.397605e-79,9.2,2.300,157.0,4.060,7.3,73.0,95.0,1.073,5.8,345.0,248.0,1.204,206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.300,73.8,5.397605e-79
3,93706.0,39.6,39.6,5.397605e-79,69.0,6099.6,5.397605e-79,57.39,47.0,1.22,148.0,3.83,NaN,NaN,NaN,NaN,NaN,NaN,6.1,24.6,9.1,61.4,4.3,0.8,1.5,0.6,3.7,0.3,5.397605e-79,5.24,16.3,47.0,89.7,31.1,34.7,12.2,233.0,6.6,5.397605e-79,0.138,...,2.7,27.0,94.0,5.22,16.0,5.397605e-79,163.0,29.2,142.0,286.0,4.0,1.292,4.4,144.0,0.7,11.97,5.397605e-79,9.6,2.400,149.0,3.853,7.1,71.0,92.0,1.039,8.0,475.8,349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.120,63.6,5.397605e-79
4,93707.0,41.6,41.6,5.397605e-79,209.0,18475.6,5.397605e-79,19.90,68.0,1.76,189.0,4.89,NaN,NaN,NaN,NaN,NaN,NaN,11.2,37.1,6.2,54.7,1.6,0.5,4.2,0.7,6.1,0.2,1.000000e-01,5.02,14.5,42.1,83.9,28.9,34.4,13.6,348.0,8.5,2.000000e-01,0.555,...,2.8,28.0,115.0,6.38,9.0,5.397605e-79,90.0,16.1,177.0,276.0,4.3,1.388,3.3,137.0,0.7,11.97,5.397605e-79,10.1,2.525,199.0,5.146,8.0,80.0,110.0,1.242,5.5,327.1,100.0,0.526,190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.483,27.4,5.397605e-79


Questionnaire Data

In [29]:
filenames_questionnaire = [i for i in glob.glob('Data/Questionnaire/*.{}'.format('XPT'))]

In [30]:
df_questionnaire = merge_file(filenames_questionnaire)

In [31]:
df_questionnaire = df_questionnaire.groupby('SEQN').max().reset_index()

In [32]:
# df_questionnaire.drop(df_questionnaire.columns[df_questionnaire.isnull().sum() > 500], axis=1, inplace=True)

In [33]:
df_questionnaire.shape

(9254, 783)

In [34]:
df_questionnaire.head()

,SEQN,ACD011A,ACD011B,ACD011C,ACD040,ACD110,AUQ054,AUQ060,AUQ070,AUQ080,AUQ090,AUQ400,AUQ410A,AUQ410B,AUQ410C,AUQ410D,AUQ410E,AUQ410F,AUQ410G,AUQ410H,AUQ410I,AUQ410J,AUQ156,AUQ420,AUQ430,AUQ139,AUQ144,AUQ147,AUQ149A,AUQ149B,AUQ149C,AUQ153,AUQ630,AUQ440,AUQ450A,AUQ450B,AUQ450C,AUQ450D,AUQ450E,AUQ450F,...,SMQ940,WHD010,WHD020,WHQ030,WHQ040,WHD050,WHQ060,WHQ070,WHD080A,WHD080B,WHD080C,WHD080D,WHD080E,WHD080F,WHD080G,WHD080H,WHD080I,WHD080J,WHD080K,WHD080M,WHD080N,WHD080O,WHD080P,WHD080Q,WHD080R,WHD080S,WHD080T,WHD080U,WHD080L,WHQ225,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200,WHQ030M,WHQ500,WHQ520
0,93703.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93705.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,63.0,165.0,1.0,3.0,165.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,150.0,130.0,63.0,170.0,62.0,2.0,NaN,NaN,NaN,NaN
3,93706.0,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,68.0,145.0,3.0,3.0,145.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,150.0,17.0,2.0,NaN,NaN,NaN,NaN
4,93707.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,3.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,6.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0


Make all sectional Data in same shape


In [35]:
os.chdir("/content/drive/My Drive/Final Project/HybridDatasets")

In [36]:
df_laboratory.shape

(8366, 190)

In [37]:
participants = list(df_laboratory['SEQN'])

In [38]:
df_laboratory.to_csv(r'LaboratoryData.csv', index = False, header=True)

In [39]:
# df_demographics.drop(df_laboratory['SEQN'], axis=0)
df_demographics = df_demographics[df_demographics['SEQN'].isin(participants)]

In [40]:
df_demographics.shape

(8366, 30)

In [41]:
df_demographics.to_csv(r'DemographicsData.csv', index = False, header=True)

In [42]:
df_dietary = df_dietary[df_dietary['SEQN'].isin(participants)]

In [43]:
df_dietary.shape

(8366, 44)

In [44]:
df_dietary.to_csv(r'DietaryData.csv', index = False, header=True)

In [45]:
df_examination = df_examination[df_examination['SEQN'].isin(participants)]

In [46]:
df_examination.shape

(8366, 138)

In [47]:
df_examination.to_csv(r'ExaminationData.csv', index = False, header=True)

In [48]:
df_questionnaire = df_questionnaire[df_questionnaire['SEQN'].isin(participants)]

In [49]:
df_questionnaire.shape

(8366, 783)

In [50]:
df_questionnaire.to_csv(r'QuestionnaireData.csv', index = False, header=True)

Medication Data

In [51]:
os.chdir("/content/drive/My Drive/Final Project")

In [52]:
df_medication = pd.read_sas('Data/Medication/RXQ_RX_J.XPT', encoding = 'utf-8')

In [53]:
df_medication = df_medication[df_medication['SEQN'].isin(participants)]

In [54]:
df_medication.head()

,SEQN,RXDUSE,RXDDRUG,RXDDRGID,RXQSEEN,RXDDAYS,RXDRSC1,RXDRSC2,RXDRSC3,RXDRSD1,RXDRSD2,RXDRSD3,RXDCOUNT
0,93703.0,2.0,,,NaN,NaN,,,,,,,NaN
1,93704.0,2.0,,,NaN,NaN,,,,,,,NaN
2,93705.0,1.0,ENALAPRIL; HYDROCHLOROTHIAZIDE,d03740,1.0,5475.0,I10,,,Essential (primary) hypertension,,,3.0
3,93705.0,1.0,MELOXICAM,d04532,1.0,61.0,M06.9,,,"Rheumatoid arthritis, unspecified",,,3.0
4,93705.0,1.0,OMEPRAZOLE,d00325,1.0,3650.0,R12,,,Heartburn,,,3.0


In [55]:
df_medication.shape

(18227, 13)

In [56]:
df_medication = df_medication.replace(r'^\s*$', np.nan, regex=True)

In [57]:
##### Column Descriptions #####
# RXDUSE: Have you taken or used any prescription medicines in the past month?
# RXDCOUNT: The number of prescription medicines taken?
# RXDDRGID: Generic drug code (Already have Drug name)
# RXDRSC1-- RXDRSC3: ICD-10-CM code (Already have description)
# RXDRSD2-- RXDRSD3: ICD-10-CM code description

unimpMedCol = ['RXDUSE', 'RXDCOUNT', 'RXDDRGID', 'RXDRSC1', 'RXDRSC2', 'RXDRSC3', 'RXDRSD2', 'RXDRSD3']

In [58]:
df_medication.drop(unimpMedCol, axis=1, inplace=True)
df_medication.head()

,SEQN,RXDDRUG,RXQSEEN,RXDDAYS,RXDRSD1
0,93703.0,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN
2,93705.0,ENALAPRIL; HYDROCHLOROTHIAZIDE,1.0,5475.0,Essential (primary) hypertension
3,93705.0,MELOXICAM,1.0,61.0,"Rheumatoid arthritis, unspecified"
4,93705.0,OMEPRAZOLE,1.0,3650.0,Heartburn


In [59]:
df_medication['SEQN'].value_counts()

100222.0    22
98585.0     20
101654.0    19
95641.0     19
98990.0     19
            ..
96677.0      1
96678.0      1
96679.0      1
102264.0     1
94208.0      1
Name: SEQN, Length: 8366, dtype: int64

In [60]:
#Create a database out of the all diseases words to get the count of frequent diseases
all_diseases_dataset = []
list_diseases = df_medication['RXDRSD1'].dropna()

for disease in list_diseases:
  disease = disease.split()

  for keyword in disease:
    keyword = keyword.lower().strip()
    all_diseases_dataset.append(keyword)

In [61]:
unique_diseasekeywords =  sorted(set(all_diseases_dataset))
len(unique_diseasekeywords)

419

In [62]:
dict_diseasekeywords = {}

for word in unique_diseasekeywords:
  dict_diseasekeywords[word] = all_diseases_dataset.count(word)

In [63]:
sorted(dict_diseasekeywords.items(), key=operator.itemgetter(1), reverse=True)

[('unspecified', 3483),
 ('(primary)', 2372),
 ('hypertension', 2370),
 ('essential', 2361),
 ('pure', 1154),
 ('hypercholesterolemia', 1122),
 ('diabetes', 1109),
 ('mellitus', 1084),
 ('type', 1084),
 ('2', 1067),
 ('and', 1038),
 ('disorder,', 991),
 ('of', 641),
 ('prevent', 583),
 ('disease', 571),
 ('depressive', 489),
 ('major', 489),
 ('episode,', 469),
 ('single', 469),
 ('gastro-esophageal', 391),
 ('reflux', 391),
 ('heart', 377),
 ('asthma', 352),
 ('hypothyroidism,', 331),
 ('anxiety', 318),
 ('blood', 315),
 ('other', 282),
 ('disease,', 278),
 ('chronic', 273),
 ('dorsalgia', 262),
 ('disorder', 255),
 ('disorders', 244),
 ('edema,', 199),
 ('pain', 199),
 ('allergic', 180),
 ('elevated', 180),
 ('glucose', 180),
 ('level', 180),
 ('in', 165),
 ('(current)', 163),
 ('long', 163),
 ('term', 163),
 ('infarction', 162),
 ('neuralgia', 161),
 ('neuritis,', 161),
 ('pulmonary', 159),
 ('attention-deficit', 153),
 ('hyperactivity', 153),
 ('dyspepsia', 151),
 ('functional', 15

In [64]:
#Top diseases that can be considered from the total diseases frequency are:
#1. Hypertension
#2. Hypercholesterolemia
#3. Diabetes mellitus
#4. Major depressive disorder - episodes
#5. Gastro-esophageal reflux disease

top_diseases = ['hypertension', 'hypercholesterolemia', 'diabetes', 'depressive', 'gastro']

In [65]:
df_medication.head()

,SEQN,RXDDRUG,RXQSEEN,RXDDAYS,RXDRSD1
0,93703.0,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN
2,93705.0,ENALAPRIL; HYDROCHLOROTHIAZIDE,1.0,5475.0,Essential (primary) hypertension
3,93705.0,MELOXICAM,1.0,61.0,"Rheumatoid arthritis, unspecified"
4,93705.0,OMEPRAZOLE,1.0,3650.0,Heartburn


In [66]:
df_medication['RXDDRUG'] = df_medication['RXDDRUG'].replace({'55555': 'Unknown', '77777': 'Unknown','99999': 'Unknown'})

In [67]:
dummy_medication = pd.get_dummies(df_medication['RXDDRUG'], prefix='DRUG')
dummy_medication.head()

,DRUG_ABACAVIR; DOLUTEGRAVIR; LAMIVUDINE,DRUG_ABACAVIR; LAMIVUDINE,DRUG_ABATACEPT,DRUG_ACARBOSE,DRUG_ACEBUTOLOL,DRUG_ACETAMINOPHEN; BUTALBITAL,DRUG_ACETAMINOPHEN; BUTALBITAL; CAFFEINE,DRUG_ACETAMINOPHEN; CODEINE,DRUG_ACETAMINOPHEN; HYDROCODONE,DRUG_ACETAMINOPHEN; OXYCODONE,DRUG_ACETAMINOPHEN; TRAMADOL,DRUG_ACETAZOLAMIDE,DRUG_ACETYLCYSTEINE,DRUG_ACYCLOVIR,DRUG_ADALIMUMAB,DRUG_ADAPALENE; BENZOYL PEROXIDE TOPICAL,DRUG_ALBIGLUTIDE,DRUG_ALBUTEROL,DRUG_ALBUTEROL; IPRATROPIUM,DRUG_ALCLOMETASONE TOPICAL,DRUG_ALENDRONATE,DRUG_ALFUZOSIN,DRUG_ALLOPURINOL,DRUG_ALOGLIPTIN,DRUG_ALOGLIPTIN; PIOGLITAZONE,DRUG_ALPRAZOLAM,DRUG_AMANTADINE,DRUG_AMBRISENTAN,DRUG_AMIODARONE,DRUG_AMITRIPTYLINE,DRUG_AMLODIPINE,DRUG_AMLODIPINE; BENAZEPRIL,DRUG_AMLODIPINE; HYDROCHLOROTHIAZIDE; OLMESARTAN,DRUG_AMLODIPINE; OLMESARTAN,DRUG_AMLODIPINE; VALSARTAN,DRUG_AMMONIUM LACTATE TOPICAL,DRUG_AMOXICILLIN,DRUG_AMOXICILLIN; CLAVULANATE,DRUG_AMPHETAMINE,DRUG_AMPHETAMINE; DEXTROAMPHETAMINE,...,DRUG_TOLTERODINE,DRUG_TOPICAL STEROIDS - UNSPECIFIED,DRUG_TOPIRAMATE,DRUG_TORSEMIDE,DRUG_TRAMADOL,DRUG_TRAVOPROST OPHTHALMIC,DRUG_TRAZODONE,DRUG_TREPROSTINIL,DRUG_TRETINOIN,DRUG_TRETINOIN TOPICAL,DRUG_TRIAMCINOLONE,DRUG_TRIAMCINOLONE TOPICAL,DRUG_TRIAMTERENE,DRUG_TRIHEXYPHENIDYL,DRUG_TRIMETHOBENZAMIDE,DRUG_TRIMETHOPRIM,DRUG_TRIPTORELIN,DRUG_TROSPIUM,DRUG_UMECLIDINIUM,DRUG_UMECLIDINIUM; VILANTEROL,DRUG_UREA TOPICAL,DRUG_URSODIOL,DRUG_USTEKINUMAB,DRUG_Unknown,DRUG_VALACYCLOVIR,DRUG_VALGANCICLOVIR,DRUG_VALPROIC ACID,DRUG_VALSARTAN,DRUG_VARENICLINE,DRUG_VENLAFAXINE,DRUG_VERAPAMIL,DRUG_VILAZODONE,DRUG_VORTIOXETINE,DRUG_WARFARIN,DRUG_ZAFIRLUKAST,DRUG_ZALEPLON,DRUG_ZIDOVUDINE,DRUG_ZIPRASIDONE,DRUG_ZOLPIDEM,DRUG_ZONISAMIDE
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
df_medication = pd.concat([df_medication, dummy_medication], axis = 1)
df_medication.drop('RXDDRUG', axis=1, inplace=True)
df_medication.head()

,SEQN,RXQSEEN,RXDDAYS,RXDRSD1,DRUG_ABACAVIR; DOLUTEGRAVIR; LAMIVUDINE,DRUG_ABACAVIR; LAMIVUDINE,DRUG_ABATACEPT,DRUG_ACARBOSE,DRUG_ACEBUTOLOL,DRUG_ACETAMINOPHEN; BUTALBITAL,DRUG_ACETAMINOPHEN; BUTALBITAL; CAFFEINE,DRUG_ACETAMINOPHEN; CODEINE,DRUG_ACETAMINOPHEN; HYDROCODONE,DRUG_ACETAMINOPHEN; OXYCODONE,DRUG_ACETAMINOPHEN; TRAMADOL,DRUG_ACETAZOLAMIDE,DRUG_ACETYLCYSTEINE,DRUG_ACYCLOVIR,DRUG_ADALIMUMAB,DRUG_ADAPALENE; BENZOYL PEROXIDE TOPICAL,DRUG_ALBIGLUTIDE,DRUG_ALBUTEROL,DRUG_ALBUTEROL; IPRATROPIUM,DRUG_ALCLOMETASONE TOPICAL,DRUG_ALENDRONATE,DRUG_ALFUZOSIN,DRUG_ALLOPURINOL,DRUG_ALOGLIPTIN,DRUG_ALOGLIPTIN; PIOGLITAZONE,DRUG_ALPRAZOLAM,DRUG_AMANTADINE,DRUG_AMBRISENTAN,DRUG_AMIODARONE,DRUG_AMITRIPTYLINE,DRUG_AMLODIPINE,DRUG_AMLODIPINE; BENAZEPRIL,DRUG_AMLODIPINE; HYDROCHLOROTHIAZIDE; OLMESARTAN,DRUG_AMLODIPINE; OLMESARTAN,DRUG_AMLODIPINE; VALSARTAN,DRUG_AMMONIUM LACTATE TOPICAL,...,DRUG_TOLTERODINE,DRUG_TOPICAL STEROIDS - UNSPECIFIED,DRUG_TOPIRAMATE,DRUG_TORSEMIDE,DRUG_TRAMADOL,DRUG_TRAVOPROST OPHTHALMIC,DRUG_TRAZODONE,DRUG_TREPROSTINIL,DRUG_TRETINOIN,DRUG_TRETINOIN TOPICAL,DRUG_TRIAMCINOLONE,DRUG_TRIAMCINOLONE TOPICAL,DRUG_TRIAMTERENE,DRUG_TRIHEXYPHENIDYL,DRUG_TRIMETHOBENZAMIDE,DRUG_TRIMETHOPRIM,DRUG_TRIPTORELIN,DRUG_TROSPIUM,DRUG_UMECLIDINIUM,DRUG_UMECLIDINIUM; VILANTEROL,DRUG_UREA TOPICAL,DRUG_URSODIOL,DRUG_USTEKINUMAB,DRUG_Unknown,DRUG_VALACYCLOVIR,DRUG_VALGANCICLOVIR,DRUG_VALPROIC ACID,DRUG_VALSARTAN,DRUG_VARENICLINE,DRUG_VENLAFAXINE,DRUG_VERAPAMIL,DRUG_VILAZODONE,DRUG_VORTIOXETINE,DRUG_WARFARIN,DRUG_ZAFIRLUKAST,DRUG_ZALEPLON,DRUG_ZIDOVUDINE,DRUG_ZIPRASIDONE,DRUG_ZOLPIDEM,DRUG_ZONISAMIDE
0,93703.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,93704.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,93705.0,1.0,5475.0,Essential (primary) hypertension,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,93705.0,1.0,61.0,"Rheumatoid arthritis, unspecified",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,93705.0,1.0,3650.0,Heartburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
dummy_diseases = pd.get_dummies(df_medication['RXDRSD1'], prefix='Disease', prefix_sep='_')
dummy_diseases.head()

,Disease_Abdominal and pelvic pain,Disease_Abnormal involuntary movements,Disease_Abnormal sputum,Disease_Abnormalities of heart beat,Disease_Acne,"Disease_Acquired absence of organs, not elsewhere classified",Disease_Acute atopic conjunctivitis,"Disease_Acute bronchitis, unspecified",Disease_Acute nasopharyngitis [common cold],Disease_Acute pharyngitis,"Disease_Acute sinusitis, unspecified","Disease_Acute upper respiratory infection, unspecified",Disease_Allergic contact dermatitis,Disease_Allergic rhinitis due to pollen,"Disease_Allergic rhinitis, unspecified","Disease_Allergy, unspecified","Disease_Alzheimer's disease, unspecified","Disease_Anemia, unspecified","Disease_Angina pectoris, unspecified","Disease_Anogenital herpesviral infection, unspecified","Disease_Anxiety disorder, unspecified",Disease_Asthma,Disease_Atherosclerosis,"Disease_Atopic dermatitis, unspecified",Disease_Attention-deficit hyperactivity disorders,"Disease_Bacterial infection, unspecified","Disease_Bipolar disorder, unspecified","Disease_Bladder disorder, unspecified","Disease_Bronchitis, not specified as acute or chronic",Disease_Calculus of kidney,Disease_Candidiasis,"Disease_Cardiac arrhythmia, unspecified",Disease_Cerebral infarction,"Disease_Chest pain, unspecified",Disease_Chronic gout,"Disease_Chronic ischemic heart disease, unspecified","Disease_Chronic kidney disease, unspecified","Disease_Chronic obstructive pulmonary disease, unspecified",Disease_Conduct disorders,Disease_Conjunctivitis,...,Disease_ST elevation (STEMI) and non-ST elevation (NSTEMI) myocardial infarction,Disease_Schizophrenia,Disease_Shortness of breath,"Disease_Sleep disorder, unspecified",Disease_Streptococcal pharyngitis,Disease_Symptoms and signs concerning food and fluid intake,Disease_Symptoms and signs involving emotional state,"Disease_Synovitis and tenosynovitis, unspecified",Disease_Testicular hypofunction,Disease_Thyroiditis,Disease_Thyrotoxicosis [hyperthyroidism],"Disease_Tremor, unspecified",Disease_Type 1 diabetes mellitus,Disease_Type 2 diabetes mellitus,Disease_Type 2 diabetes mellitus with kidney complications,Disease_Type 2 diabetes mellitus with neurological complications,Disease_Type 2 diabetes mellitus with unspecified complications,Disease_Ulcerative colitis,Disease_Unspecified abdominal hernia,Disease_Unspecified abnormalities of breathing,Disease_Unspecified atrial fibrillation and atrial flutter,Disease_Unspecified cataract,Disease_Unspecified chronic bronchitis,Disease_Unspecified disorder of circulatory system,Disease_Unspecified disorder of ear,Disease_Unspecified disorder of eye and adnexa,Disease_Unspecified disorder of nose and nasal sinuses,Disease_Unspecified infectious disease,Disease_Unspecified injury,Disease_Unspecified kidney failure,Disease_Unspecified mood [affective] disorder,Disease_Unspecified psychosis not due to a substance or known physiological condition,Disease_Unspecified symptoms and signs involving the genitourinary system,Disease_Unspecified urinary incontinence,"Disease_Urinary tract infection, site not specified",Disease_Urticaria,"Disease_Viral infection, unspecified",Disease_Wheezing,Disease_Zoster [herpes zoster],Disease_prevent breast cancer
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [70]:
# df_medication = df_medication[df_medication.RXDRSD1.str.contains('|'.join(top_diseases), case=False)]
final_diseases = [disease for disease in list(dummy_diseases.columns) if any(key_disease.lower() in disease.lower() for key_disease in top_diseases)]

In [71]:
final_diseases

['Disease_Essential (primary) hypertension',
 'Disease_Gastro-esophageal reflux disease',
 'Disease_Major depressive disorder, recurrent, unspecified',
 'Disease_Major depressive disorder, single episode, unspecified',
 'Disease_Prevent diabetes',
 'Disease_Prevent hypertension',
 'Disease_Pure hypercholesterolemia',
 'Disease_Type 1 diabetes mellitus',
 'Disease_Type 2 diabetes mellitus',
 'Disease_Type 2 diabetes mellitus with kidney complications',
 'Disease_Type 2 diabetes mellitus with neurological complications',
 'Disease_Type 2 diabetes mellitus with unspecified complications']

In [72]:
# dummy_diseases[final_diseases]
df_medication = pd.concat([df_medication, dummy_diseases[final_diseases]], axis = 1)
df_medication.drop('RXDRSD1', axis=1, inplace=True)
df_medication.head()

,SEQN,RXQSEEN,RXDDAYS,DRUG_ABACAVIR; DOLUTEGRAVIR; LAMIVUDINE,DRUG_ABACAVIR; LAMIVUDINE,DRUG_ABATACEPT,DRUG_ACARBOSE,DRUG_ACEBUTOLOL,DRUG_ACETAMINOPHEN; BUTALBITAL,DRUG_ACETAMINOPHEN; BUTALBITAL; CAFFEINE,DRUG_ACETAMINOPHEN; CODEINE,DRUG_ACETAMINOPHEN; HYDROCODONE,DRUG_ACETAMINOPHEN; OXYCODONE,DRUG_ACETAMINOPHEN; TRAMADOL,DRUG_ACETAZOLAMIDE,DRUG_ACETYLCYSTEINE,DRUG_ACYCLOVIR,DRUG_ADALIMUMAB,DRUG_ADAPALENE; BENZOYL PEROXIDE TOPICAL,DRUG_ALBIGLUTIDE,DRUG_ALBUTEROL,DRUG_ALBUTEROL; IPRATROPIUM,DRUG_ALCLOMETASONE TOPICAL,DRUG_ALENDRONATE,DRUG_ALFUZOSIN,DRUG_ALLOPURINOL,DRUG_ALOGLIPTIN,DRUG_ALOGLIPTIN; PIOGLITAZONE,DRUG_ALPRAZOLAM,DRUG_AMANTADINE,DRUG_AMBRISENTAN,DRUG_AMIODARONE,DRUG_AMITRIPTYLINE,DRUG_AMLODIPINE,DRUG_AMLODIPINE; BENAZEPRIL,DRUG_AMLODIPINE; HYDROCHLOROTHIAZIDE; OLMESARTAN,DRUG_AMLODIPINE; OLMESARTAN,DRUG_AMLODIPINE; VALSARTAN,DRUG_AMMONIUM LACTATE TOPICAL,DRUG_AMOXICILLIN,...,DRUG_TRIAMTERENE,DRUG_TRIHEXYPHENIDYL,DRUG_TRIMETHOBENZAMIDE,DRUG_TRIMETHOPRIM,DRUG_TRIPTORELIN,DRUG_TROSPIUM,DRUG_UMECLIDINIUM,DRUG_UMECLIDINIUM; VILANTEROL,DRUG_UREA TOPICAL,DRUG_URSODIOL,DRUG_USTEKINUMAB,DRUG_Unknown,DRUG_VALACYCLOVIR,DRUG_VALGANCICLOVIR,DRUG_VALPROIC ACID,DRUG_VALSARTAN,DRUG_VARENICLINE,DRUG_VENLAFAXINE,DRUG_VERAPAMIL,DRUG_VILAZODONE,DRUG_VORTIOXETINE,DRUG_WARFARIN,DRUG_ZAFIRLUKAST,DRUG_ZALEPLON,DRUG_ZIDOVUDINE,DRUG_ZIPRASIDONE,DRUG_ZOLPIDEM,DRUG_ZONISAMIDE,Disease_Essential (primary) hypertension,Disease_Gastro-esophageal reflux disease,"Disease_Major depressive disorder, recurrent, unspecified","Disease_Major depressive disorder, single episode, unspecified",Disease_Prevent diabetes,Disease_Prevent hypertension,Disease_Pure hypercholesterolemia,Disease_Type 1 diabetes mellitus,Disease_Type 2 diabetes mellitus,Disease_Type 2 diabetes mellitus with kidney complications,Disease_Type 2 diabetes mellitus with neurological complications,Disease_Type 2 diabetes mellitus with unspecified complications
0,93703.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,93704.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,93705.0,1.0,5475.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,93705.0,1.0,61.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,93705.0,1.0,3650.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [73]:
df_medication.fillna(0, inplace = True)

In [74]:
df_medication['Disease_diabetes'] = df_medication['Disease_Prevent diabetes'] | df_medication['Disease_Type 1 diabetes mellitus'] | df_medication['Disease_Type 2 diabetes mellitus'] | df_medication['Disease_Type 2 diabetes mellitus with kidney complications'] | df_medication['Disease_Type 2 diabetes mellitus with neurological complications'] | df_medication['Disease_Type 2 diabetes mellitus with unspecified complications']
df_medication.drop(['Disease_Prevent diabetes', 'Disease_Type 1 diabetes mellitus', 'Disease_Type 2 diabetes mellitus', 'Disease_Type 2 diabetes mellitus with kidney complications', 'Disease_Type 2 diabetes mellitus with neurological complications', 'Disease_Type 2 diabetes mellitus with unspecified complications'], axis=1, inplace=True)

In [75]:
df_medication['Disease_hypertension'] = df_medication['Disease_Essential (primary) hypertension'] | df_medication['Disease_Prevent hypertension']
df_medication.drop(['Disease_Essential (primary) hypertension', 'Disease_Prevent hypertension'],axis=1, inplace=True)

In [76]:
df_medication['Disease_depressive disorder'] = df_medication['Disease_Major depressive disorder, recurrent, unspecified'] | df_medication['Disease_Major depressive disorder, single episode, unspecified']
df_medication.drop(['Disease_Major depressive disorder, recurrent, unspecified', 'Disease_Major depressive disorder, single episode, unspecified'], axis=1, inplace=True)

In [77]:
df_medication.shape

(18227, 670)

In [78]:
df_medication_drugs = df_medication.iloc[:,0:197]

Individual merged datasets with respect to each disease target value.

In [79]:
def manage_duplicates(dataset, target):
  new_data = dataset.groupby(['SEQN', target], sort=True).max().reset_index()
  new_data = new_data.drop_duplicates(subset=['SEQN'], keep='last')
  new_data = new_data.loc[:,(new_data != 0).any(axis=0)]
  return new_data

In [80]:
os.chdir("/content/drive/My Drive/COMP-5800-YDK-2020_Group1/HybridDatasets")

Hypertension Data

In [81]:
df_hypertension = pd.concat([df_medication_drugs, df_medication['Disease_hypertension']], axis=1)
df_hypertension = df_hypertension.reset_index(drop=True)

In [82]:
df_hypertension = manage_duplicates(df_hypertension,'Disease_hypertension')

In [83]:
df_hypertension.shape

(8366, 178)

In [84]:
df_hypertension.rename(columns = {'Disease_hypertension' : 'Target'}, inplace = True)

In [85]:
df_hypertension.to_csv(r'HypertensionData.csv', index = False, header=True)

Diabetes Data

In [86]:
df_diabetes = pd.concat([df_medication_drugs, df_medication['Disease_diabetes']], axis=1)
df_diabetes = df_diabetes.reset_index(drop=True)

In [87]:
df_diabetes = manage_duplicates(df_diabetes,'Disease_diabetes')

In [88]:
df_diabetes.shape

(8366, 191)

In [89]:
df_diabetes.rename(columns = {'Disease_diabetes' : 'Target'}, inplace = True)

In [90]:
df_diabetes.to_csv(r'DiabetesData.csv', index = False, header=True)

Hypercholesterolemia dataset

In [91]:
df_hypercholesterolemia = pd.concat([df_medication_drugs, df_medication['Disease_Pure hypercholesterolemia']], axis=1)
df_hypercholesterolemia = df_hypercholesterolemia.reset_index(drop=True)

In [92]:
df_hypercholesterolemia = manage_duplicates(df_hypercholesterolemia, 'Disease_Pure hypercholesterolemia')

In [93]:
df_hypercholesterolemia.shape

(8366, 169)

In [94]:
df_hypercholesterolemia.rename(columns = {'Disease_Pure hypercholesterolemia' : 'Target'}, inplace = True)

In [95]:
df_hypercholesterolemia.to_csv(r'HypercholesterolemiaData.csv', index = False, header=True)

Depressive disorder Data

In [96]:
df_depressive_disorder = pd.concat([df_medication_drugs, df_medication['Disease_depressive disorder']], axis=1)
df_depressive_disorder = df_depressive_disorder.reset_index(drop=True)

In [97]:
df_depressive_disorder = manage_duplicates(df_depressive_disorder, 'Disease_depressive disorder')

In [98]:
df_depressive_disorder.shape

(8366, 185)

In [99]:
df_depressive_disorder.rename(columns = {'Disease_depressive disorder' : 'Target'}, inplace = True)

In [100]:
df_depressive_disorder.to_csv(r'DepressiveDisorderData.csv', index = False, header=True)

Gastro-esophageal reflux disease data

In [101]:
df_Gastro_esophageal = pd.concat([df_medication_drugs, df_medication['Disease_Gastro-esophageal reflux disease']], axis=1)
df_Gastro_esophageal = df_Gastro_esophageal.reset_index(drop=True)

In [102]:
df_Gastro_esophageal = manage_duplicates(df_Gastro_esophageal, 'Disease_Gastro-esophageal reflux disease')

In [103]:
df_Gastro_esophageal.shape

(8366, 193)

In [104]:
df_Gastro_esophageal.rename(columns = {'Disease_Gastro-esophageal reflux disease' : 'Target'}, inplace = True)

In [105]:
df_Gastro_esophageal.to_csv(r'GastroEsophagealData.csv', index = False, header=True)

Impute the values in Datasets

In [106]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

In [107]:
os.chdir("/content/drive/My Drive/COMP-5800-YDK-2020_Group1")

In [108]:
df_demographics = pd.DataFrame(imputer.fit_transform(df_demographics),columns = df_demographics.columns)

In [109]:
df_demographics.to_csv(r'ImputedDataset/DemographicsData.csv', index = False, header=True)

In [110]:
df_dietary = pd.DataFrame(imputer.fit_transform(df_dietary),columns = df_dietary.columns)

In [111]:
df_dietary.to_csv(r'ImputedDataset/DietaryData.csv', index = False, header=True)

In [112]:
df_examination

,SEQN,BPXCHR,BPXPULS,BMDSTATS,BMXWT,OHDEXSTS,OHDRCSTS,OHDDESTS,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,OHX11CTC,OHX12CTC,OHX13CTC,OHX14CTC,OHX15CTC,OHX18CTC,OHX19CTC,OHX20CTC,OHX21CTC,OHX22CTC,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC,OHX02CSC,OHX03CSC,OHX04CSC,OHX05CSC,...,OHX08RSC,OHX09RSC,OHX10RSC,OHX11RSC,OHX12RSC,OHX13RSC,OHX14RSC,OHX15RSC,OHX18RSC,OHX19RSC,OHX20RSC,OHX21RSC,OHX22RSC,OHX23RSC,OHX24RSC,OHX25RSC,OHX26RSC,OHX27RSC,OHX28RSC,OHX29RSC,OHX30RSC,OHX31RSC,OHX02SE,OHX03SE,OHX04SE,OHX05SE,OHX07SE,OHX10SE,OHX12SE,OHX13SE,OHX14SE,OHX15SE,OHX18SE,OHX19SE,OHX20SE,OHX21SE,OHX28SE,OHX29SE,OHX30SE,OHX31SE
0,93703.0,120.0,1.0,1.0,13.7,1.0,1.0,1.0,U,U,D,D,D,D,D,D,D,D,D,D,U,U,U,U,D,D,D,D,D,D,D,D,D,D,U,U,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,93704.0,114.0,1.0,1.0,13.9,1.0,1.0,1.0,U,U,D,D,D,D,D,D,D,D,D,D,U,U,U,U,D,D,D,D,D,D,D,D,D,D,U,U,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,93705.0,52.0,1.0,1.0,79.5,1.0,1.0,1.0,F,E,F,F,S,S,S,Y,S,S,F,F,F,F,F,R,F,F,S,S,S,S,S,S,F,F,R,F,1,,1,1,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,93706.0,82.0,1.0,1.0,66.3,1.0,1.0,1.0,S,F,S,S,S,S,S,S,S,S,S,S,S,S,S,F,S,S,S,S,S,S,S,S,S,S,S,S,,1,,,...,,,,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,93707.0,100.0,1.0,1.0,45.4,1.0,1.0,1.0,U,S,D,D,D,S,S,S,S,D,D,D,S,U,U,S,D,S,S,S,S,S,S,S,D,S,S,S,,,,,...,,,,,,,,,,,,,,,,,,,,,,,9,0,0,0,0,0,0,0,0,9,9,0,0,9,0,9,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699,102952.0,68.0,1.0,1.0,49.0,1.0,1.0,1.0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8700,102953.0,78.0,1.0,1.0,97.4,1.0,1.0,1.0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,Z,S,S,S,S,S,S,S,S,S,S,S,Z,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8701,102954.0,78.0,1.0,1.0,69.1,1.0,1.0,1.0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,F,S,S,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8702,102955.0,74.0,1.0,1.0,111.9,1.0,1.0,1.0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,Z,,,,,...,,,,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [113]:
exam_convert = df_examination.columns[df_examination.dtypes == object]

In [114]:
df_examination[exam_convert] = np.asarray(df_examination[exam_convert]).astype(str)

In [115]:
df_examination[exam_convert] = df_examination[exam_convert].replace(np.nan, -1)

In [116]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [117]:
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [118]:
onehot = OneHotEncoder(handle_unknown = 'ignore')
df_examination[exam_convert] = df_examination[exam_convert].apply(lambda x: d[x.name].fit_transform(x))

In [119]:
# df_examination[exam_convert] = pd.DataFrame(imputer.fit_transform(df_examination[exam_convert]),columns = df_examination[exam_convert].columns)

In [120]:
df_examination

,SEQN,BPXCHR,BPXPULS,BMDSTATS,BMXWT,OHDEXSTS,OHDRCSTS,OHDDESTS,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,OHX11CTC,OHX12CTC,OHX13CTC,OHX14CTC,OHX15CTC,OHX18CTC,OHX19CTC,OHX20CTC,OHX21CTC,OHX22CTC,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC,OHX02CSC,OHX03CSC,OHX04CSC,OHX05CSC,...,OHX08RSC,OHX09RSC,OHX10RSC,OHX11RSC,OHX12RSC,OHX13RSC,OHX14RSC,OHX15RSC,OHX18RSC,OHX19RSC,OHX20RSC,OHX21RSC,OHX22RSC,OHX23RSC,OHX24RSC,OHX25RSC,OHX26RSC,OHX27RSC,OHX28RSC,OHX29RSC,OHX30RSC,OHX31RSC,OHX02SE,OHX03SE,OHX04SE,OHX05SE,OHX07SE,OHX10SE,OHX12SE,OHX13SE,OHX14SE,OHX15SE,OHX18SE,OHX19SE,OHX20SE,OHX21SE,OHX28SE,OHX29SE,OHX30SE,OHX31SE
0,93703.0,120.0,1.0,1.0,13.7,1.0,1.0,1.0,9,10,2,2,2,2,2,2,2,2,2,2,10,9,9,11,2,2,2,2,2,2,2,2,2,2,10,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,93704.0,114.0,1.0,1.0,13.9,1.0,1.0,1.0,9,10,2,2,2,2,2,2,2,2,2,2,10,9,9,11,2,2,2,2,2,2,2,2,2,2,10,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,93705.0,52.0,1.0,1.0,79.5,1.0,1.0,1.0,2,2,4,4,11,11,11,15,11,11,4,4,3,2,2,8,4,4,11,11,11,10,11,11,4,4,8,2,36,0,34,28,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,93706.0,82.0,1.0,1.0,66.3,1.0,1.0,1.0,8,3,11,11,11,11,11,11,11,11,11,11,9,8,8,3,11,11,11,11,11,10,11,11,11,11,9,8,0,42,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,93707.0,100.0,1.0,1.0,45.4,1.0,1.0,1.0,9,9,2,2,2,11,11,11,11,2,2,2,9,9,9,9,2,11,11,11,11,10,11,11,2,11,9,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,1,1,1,1,1,1,1,4,4,1,1,4,1,4,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699,102952.0,68.0,1.0,1.0,49.0,1.0,1.0,1.0,8,9,11,11,11,11,11,11,11,11,11,11,9,8,8,9,11,11,11,11,11,10,11,11,11,11,9,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8700,102953.0,78.0,1.0,1.0,97.4,1.0,1.0,1.0,8,9,11,11,11,11,11,11,11,11,11,11,9,8,8,14,11,11,11,11,11,10,11,11,11,11,9,11,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8701,102954.0,78.0,1.0,1.0,69.1,1.0,1.0,1.0,8,9,11,11,11,11,11,11,11,11,11,11,9,8,8,9,11,11,11,11,11,10,11,11,11,4,9,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8702,102955.0,74.0,1.0,1.0,111.9,1.0,1.0,1.0,8,9,11,11,11,11,11,11,11,11,11,11,9,8,8,9,11,11,11,11,11,10,11,11,11,11,9,11,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [121]:
df_examination = df_examination.fillna(df_examination.mean())

In [122]:
df_examination.to_csv(r'ImputedDataset/ExaminationData.csv', index = False, header=True)

In [123]:
df_laboratory = pd.DataFrame(imputer.fit_transform(df_laboratory),columns = df_laboratory.columns)

In [124]:
df_laboratory.to_csv(r'ImputedDataset/LaboratoryData.csv', index = False, header=True)

In [125]:
df_questionnaire.drop(df_questionnaire.columns[df_questionnaire.dtypes == object], axis = 1, inplace=True)

In [126]:
# df_questionnaire = pd.DataFrame(imputer.fit_transform(df_questionnaire),columns = df_questionnaire.columns)

In [127]:
df_questionnaire.to_csv(r'ImputedDataset/QuesionnaireData.csv', index = False, header=True)